In [1]:
# qc_forecast.py
# -------------------------------------------------------------
# End-to-end pipeline to forecast store×product refurb units
# to pre-stock for the next H days, using MVP (moving average) 
# and PRO (lifecycle bucket hazard) approaches.
#
# Usage:
#   1) Edit PATHS to point to your 4 CSV files
#   2) Adjust PARAMS if needed  
#   3) Run: python qc_forecast.py
#
# Outputs 4 CSV files:
#   - qc_hotspots.csv (risk ranking)
#   - qc_mvp_forecast.csv (moving average forecast) 
#   - qc_pro_forecast.csv (lifecycle bucket forecast)
#   - qc_final_forecast.csv (blended recommendation)
# -------------------------------------------------------------

import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
from typing import Dict

# ---------------------------
# PARAMETERS (Tune as needed)
# ---------------------------
PARAMS = {
    "HORIZON_DAYS": 30,          # Planning horizon (days)
    "K_MONTHS_MA": 3,            # Moving average window (months)
    "LEAD_TIME_DAYS": 14,        # Replenishment lead time 
    "SERVICE_Z": 1.65,           # 95% service level (1.65), 99%→2.33
    "LOOKBACK_DAYS_HOTSPOT": 180,# Hotspot analysis window
    "BUCKET_DAYS": [0, 180, 360],# Lifecycle buckets: 0-180, 181-360, 361+
    "BLEND_WEIGHT_PRO": 0.7,     # Final blend: 70% PRO + 30% MVP
    "MIN_HISTORY_CLAIMS": 30     # Sparse history threshold
}

# ---------------------------
# Utilities
# ---------------------------
def to_date(s):
    """Convert to date, handle errors"""
    return pd.to_datetime(s, errors="coerce").dt.date

def days_between(a, b):
    """Days between two date series"""
    return (pd.to_datetime(a) - pd.to_datetime(b)).dt.days

def safe_div(a, b):
    """Safe division"""
    return np.where(b == 0, 0.0, a / b)

def ensure_nonneg_int(x):
    """Ensure non-negative integers"""
    return np.maximum(0, np.round(x).astype(int))

# ---------------------------
# Data Loading & Validation
# ---------------------------
def load_and_validate_data(paths):
    """Load and validate 4 CSV files"""
    
    # Check files exist
    for name, path in paths.items():
        if not os.path.exists(path):
            raise FileNotFoundError(f"File not found: {path}")
    
    # Load data
    products = pd.read_csv(paths["products"])
    stores = pd.read_csv(paths["stores"])
    sales = pd.read_csv(paths["sales"])
    warranty = pd.read_csv(paths["warranty"])
    
    # Required columns check
    req_cols = {
        "products": ["product_id", "product_name", "category_id"],
        "stores": ["store_id", "store_name", "city", "country"], 
        "sales": ["sale_id", "sale_date", "store_id", "product_id", "quantity"],
        "warranty": ["claim_id", "claim_date", "sale_id"]
    }
    
    for name, cols in req_cols.items():
        df = locals()[name]
        missing = [c for c in cols if c not in df.columns]
        if missing:
            raise ValueError(f"Missing columns in {name}.csv: {missing}")
    
    # Convert dates and clean
    if "launch_date" in products.columns:
        products["launch_date"] = to_date(products["launch_date"])
    sales["sale_date"] = to_date(sales["sale_date"])
    warranty["claim_date"] = to_date(warranty["claim_date"])
    
    # Basic cleaning
    sales = sales[sales["quantity"] > 0].dropna(subset=["sale_date"])
    warranty = warranty.dropna(subset=["claim_date"])
    sales = sales.drop_duplicates(subset=["sale_id"])
    warranty = warranty.drop_duplicates(subset=["claim_id"])
    
    # Calculate TODAY
    today = max(
        sales["sale_date"].max() if not sales.empty else datetime.now().date(),
        warranty["claim_date"].max() if not warranty.empty else datetime.now().date()
    )
    
    print(f" Data loaded. Sales: {len(sales):,}, Claims: {len(warranty):,}, TODAY: {today}")
    return products, stores, sales, warranty, today

# ---------------------------
# A) Hotspot Ranking
# ---------------------------
def build_hotspots(products, stores, sales, warranty, today, params):
    """Recent claim probability ranking by store×product"""
    
    cutoff = today - timedelta(days=params["LOOKBACK_DAYS_HOTSPOT"])
    
    # Recent sales
    recent_sales = sales[sales["sale_date"] >= cutoff].copy()
    if recent_sales.empty:
        print("No recent sales for hotspot analysis")
        return pd.DataFrame()
    
    # Aggregate sales by store×product
    base = (recent_sales
            .merge(products[["product_id", "product_name"]], on="product_id")
            .merge(stores[["store_id", "store_name", "country"]], on="store_id")
            .groupby(["country", "store_name", "product_id", "product_name"])
            .agg(units_sold=("quantity", "sum"))
            .reset_index())
    
    # Recent claims
    recent_claims = warranty[warranty["claim_date"] >= cutoff].copy()
    if not recent_claims.empty:
        claims_agg = (recent_claims
                     .merge(sales[["sale_id", "store_id", "product_id"]], on="sale_id")
                     .merge(products[["product_id", "product_name"]], on="product_id")
                     .merge(stores[["store_id", "store_name", "country"]], on="store_id")
                     .groupby(["country", "store_name", "product_id", "product_name"])
                     .agg(claims=("claim_id", "count"))
                     .reset_index())
        
        df = base.merge(claims_agg, on=["country", "store_name", "product_id", "product_name"], how="left")
    else:
        df = base.copy()
    
    df["claims"] = df["claims"].fillna(0).astype(int)
    df = df[df["units_sold"] > 0]
    df["claim_probability_pct"] = np.round(100.0 * df["claims"] / df["units_sold"], 2)
    df["risk_rank"] = df["claim_probability_pct"].rank(method="min", ascending=False).astype(int)
    
    return df.sort_values(["risk_rank", "country", "store_name", "product_name"])

# ---------------------------
# B) MVP Forecast 
# ---------------------------
def mvp_forecast(products, stores, sales, warranty, today, params):
    """Moving average + safety stock forecast"""
    
    if warranty.empty:
        print("No warranty data for MVP forecast")
        return pd.DataFrame()
    
    # Monthly claims aggregation
    ws = (warranty
          .merge(sales[["sale_id", "store_id", "product_id"]], on="sale_id")
          .merge(products[["product_id", "product_name"]], on="product_id")
          .merge(stores[["store_id", "store_name", "country"]], on="store_id"))
    
    ws["month"] = pd.to_datetime(ws["claim_date"]).dt.to_period("M")
    
    monthly_claims = (ws
                     .groupby(["country", "store_name", "product_id", "product_name", "month"])
                     .agg(claims=("claim_id", "count"))
                     .reset_index())
    
    # Keep last K months
    cutoff_period = pd.Timestamp(today).to_period("M") - params["K_MONTHS_MA"]
    recent_claims = monthly_claims[monthly_claims["month"] >= cutoff_period]
    
    if recent_claims.empty:
        print("No recent claims for MVP forecast")
        return pd.DataFrame()
    
    # Statistics per store×product
    stats = (recent_claims
             .groupby(["country", "store_name", "product_id", "product_name"])["claims"]
             .agg(mean_monthly="mean", std_monthly="std")
             .reset_index())
    
    stats["std_monthly"] = stats["std_monthly"].fillna(0.0)
    
    # Convert to horizon period
    horizon_days = params["HORIZON_DAYS"]
    lead_time = params["LEAD_TIME_DAYS"] 
    service_z = params["SERVICE_Z"]
    
    stats["demand_horizon"] = ensure_nonneg_int(np.ceil(stats["mean_monthly"] * (horizon_days / 30.0)))
    stats["safety_stock"] = ensure_nonneg_int(np.ceil(service_z * stats["std_monthly"] * np.sqrt(lead_time / 30.0)))
    stats["mvp_units_to_prestore"] = stats["demand_horizon"] + stats["safety_stock"]
    
    # Add dates
    stats["horizon_end"] = today + timedelta(days=horizon_days)
    stats["latest_stock_date"] = today + timedelta(days=horizon_days - lead_time)
    
    return stats.sort_values("mvp_units_to_prestore", ascending=False)

# ---------------------------
# C) PRO Forecast
# ---------------------------
def calculate_installed_base(stores, products, sales, warranty):
    """Calculate installed base = sold - claimed per store×product"""
    
    # Total sales
    sold = (sales
            .groupby(["store_id", "product_id"])
            .agg(units_sold=("quantity", "sum"))
            .reset_index())
    
    # Total claims
    if not warranty.empty:
        claimed = (warranty
                  .merge(sales[["sale_id", "store_id", "product_id"]], on="sale_id")
                  .groupby(["store_id", "product_id"])
                  .agg(units_claimed=("claim_id", "count"))
                  .reset_index())
    else:
        claimed = pd.DataFrame(columns=["store_id", "product_id", "units_claimed"])
    
    # Merge and calculate installed base
    base = sold.merge(claimed, on=["store_id", "product_id"], how="left")
    base["units_claimed"] = base["units_claimed"].fillna(0)
    base["installed_base"] = (base["units_sold"] - base["units_claimed"]).clip(lower=0)
    
    # Add store and product info
    base = (base
            .merge(stores[["store_id", "store_name", "country"]], on="store_id")
            .merge(products[["product_id", "product_name", "category_id", "launch_date"]], on="product_id"))
    
    return base

def calculate_product_hazards(products, sales, warranty, params):
    """Calculate product-level claim probability and lifecycle bucket hazards"""
    
    # Overall product claim rates
    product_sales = sales.groupby("product_id").agg(total_sold=("quantity", "sum")).reset_index()
    
    if not warranty.empty:
        product_claims = (warranty
                         .merge(sales[["sale_id", "product_id"]], on="sale_id")
                         .groupby("product_id")
                         .agg(total_claims=("claim_id", "count"))
                         .reset_index())
        
        product_rates = product_sales.merge(product_claims, on="product_id", how="left")
        product_rates["total_claims"] = product_rates["total_claims"].fillna(0)
        product_rates["claim_prob"] = safe_div(product_rates["total_claims"], product_rates["total_sold"])
    else:
        product_rates = product_sales.copy()
        product_rates["total_claims"] = 0
        product_rates["claim_prob"] = 0.0
    
    # Lifecycle bucket analysis
    if not warranty.empty:
        bucket_claims = (warranty
                        .merge(sales[["sale_id", "product_id", "sale_date"]], on="sale_id")
                        .merge(products[["product_id", "launch_date"]], on="product_id"))
        
        bucket_claims["age_days"] = days_between(bucket_claims["claim_date"], bucket_claims["launch_date"])
        
        def assign_bucket(days):
            if pd.isna(days) or days < 0:
                return "unknown"
            elif days <= params["BUCKET_DAYS"][1]:  # 0-180
                return "bucket_0_6"
            elif days <= params["BUCKET_DAYS"][2]:  # 181-360
                return "bucket_7_12" 
            else:
                return "bucket_13p"
        
        bucket_claims["bucket"] = bucket_claims["age_days"].apply(assign_bucket)
        
        bucket_pivot = (bucket_claims
                       .groupby(["product_id", "bucket"])
                       .agg(claims=("claim_id", "count"))
                       .reset_index()
                       .pivot_table(index="product_id", columns="bucket", values="claims", fill_value=0)
                       .reset_index())
        
        # Merge with product rates
        hazards = product_rates.merge(bucket_pivot, on="product_id", how="left").fillna(0)
    else:
        hazards = product_rates.copy()
        for bucket in ["bucket_0_6", "bucket_7_12", "bucket_13p"]:
            hazards[bucket] = 0
    
    # Calculate bucket shares and monthly rates
    bucket_cols = ["bucket_0_6", "bucket_7_12", "bucket_13p"]
    hazards["total_bucket_claims"] = hazards[bucket_cols].sum(axis=1)
    
    for bucket in bucket_cols:
        if hazards["total_bucket_claims"].sum() > 0:
            hazards[f"{bucket}_share"] = safe_div(hazards[bucket], hazards["total_bucket_claims"])
        else:
            hazards[f"{bucket}_share"] = 1/3  # Equal split if no history
    
    # Monthly hazard rates (claim_prob × bucket_share / months_in_bucket)
    hazards["rate_0_6"] = hazards["claim_prob"] * hazards["bucket_0_6_share"] / 6.0
    hazards["rate_7_12"] = hazards["claim_prob"] * hazards["bucket_7_12_share"] / 6.0  
    hazards["rate_13p"] = hazards["claim_prob"] * hazards["bucket_13p_share"] / 999.0
    
    return hazards

def pro_forecast(products, stores, sales, warranty, today, params):
    """Lifecycle bucket hazard × installed base forecast"""
    
    # Get installed base and hazard rates
    installed_base = calculate_installed_base(stores, products, sales, warranty)
    hazards = calculate_product_hazards(products, sales, warranty, params)
    
    if installed_base.empty:
        print("No installed base for PRO forecast")
        return pd.DataFrame()
    
    # Determine current age bucket for each store×product
    installed_base["age_days"] = days_between(today, installed_base["launch_date"])
    
    def current_bucket(days):
        if pd.isna(days) or days < 0:
            return "bucket_13p"
        elif days <= params["BUCKET_DAYS"][1]:
            return "bucket_0_6"
        elif days <= params["BUCKET_DAYS"][2]:
            return "bucket_7_12"
        else:
            return "bucket_13p"
    
    installed_base["current_bucket"] = installed_base["age_days"].apply(current_bucket)
    
    # Merge with hazard rates
    forecast = installed_base.merge(hazards[["product_id", "rate_0_6", "rate_7_12", "rate_13p", "total_claims"]], on="product_id")
    
    # Apply conservative factor for sparse history
    sparse_mask = forecast["total_claims"] < params["MIN_HISTORY_CLAIMS"]
    for rate_col in ["rate_0_6", "rate_7_12", "rate_13p"]:
        forecast.loc[sparse_mask, rate_col] *= 1.25
    
    # Calculate expected claims for horizon period
    horizon_days = params["HORIZON_DAYS"]
    horizon_factor = horizon_days / 30.0
    
    forecast["rate"] = np.where(forecast["current_bucket"] == "bucket_0_6", forecast["rate_0_6"],
                       np.where(forecast["current_bucket"] == "bucket_7_12", forecast["rate_7_12"], 
                               forecast["rate_13p"]))
    
    forecast["expected_claims"] = ensure_nonneg_int(np.ceil(forecast["installed_base"] * forecast["rate"] * horizon_factor))
    
    # Safety stock (simplified - using global variance)
    global_std = 0.5  # Conservative assumption
    service_z = params["SERVICE_Z"]
    lead_time = params["LEAD_TIME_DAYS"]
    forecast["safety_stock"] = ensure_nonneg_int(np.ceil(service_z * global_std * np.sqrt(lead_time / 30.0)))
    
    forecast["pro_units_to_prestore"] = forecast["expected_claims"] + forecast["safety_stock"]
    
    # Add dates
    forecast["horizon_end"] = today + timedelta(days=horizon_days)
    forecast["latest_stock_date"] = today + timedelta(days=horizon_days - params["LEAD_TIME_DAYS"])
    
    return forecast[["country", "store_name", "product_id", "product_name", 
                    "expected_claims", "safety_stock", "pro_units_to_prestore",
                    "horizon_end", "latest_stock_date"]].sort_values("pro_units_to_prestore", ascending=False)

# ---------------------------
# D) Final Blending & Output
# ---------------------------
def blend_and_save(hotspots, mvp, pro, params, outdir="."):
    """Blend MVP and PRO forecasts and save all outputs"""
    
    w_pro = params["BLEND_WEIGHT_PRO"]
    keys = ["country", "store_name", "product_id", "product_name"]
    
    # Handle empty forecasts
    if mvp.empty and pro.empty:
        print("Both forecasts empty")
        final = pd.DataFrame()
    elif mvp.empty:
        final = pro.copy()
        final["final_units_to_prestore"] = final["pro_units_to_prestore"]
    elif pro.empty:
        final = mvp.copy() 
        final["final_units_to_prestore"] = final["mvp_units_to_prestore"]
    else:
        # Merge and blend
        final = pro.merge(mvp, on=keys, how="outer", suffixes=("", "_mvp"))
        final["pro_units_to_prestore"] = final["pro_units_to_prestore"].fillna(0)
        final["mvp_units_to_prestore"] = final["mvp_units_to_prestore"].fillna(0)
        
        final["final_units_to_prestore"] = ensure_nonneg_int(
            w_pro * final["pro_units_to_prestore"] + (1-w_pro) * final["mvp_units_to_prestore"]
        )
        
        # Use PRO dates if available
        final["horizon_end"] = final["horizon_end"].fillna(final.get("horizon_end_mvp"))
        final["latest_stock_date"] = final["latest_stock_date"].fillna(final.get("latest_stock_date_mvp"))
    
    # Add hotspot ranking
    if not hotspots.empty and not final.empty:
        final = final.merge(hotspots[keys + ["claim_probability_pct", "risk_rank"]], on=keys, how="left")
    
    # Sort by priority
    if not final.empty:
        sort_cols = ["final_units_to_prestore"]
        if "risk_rank" in final.columns:
            sort_cols.append("risk_rank") 
        sort_cols.extend(["country", "store_name", "product_name"])
        
        ascending = [False, True, True, True, True][:len(sort_cols)]
        final = final.sort_values(sort_cols, ascending=ascending)
    
    # Save outputs
    try:
        if not hotspots.empty:
            hotspots.to_csv(f"{outdir}/qc_hotspots.csv", index=False)
            print(f"Saved qc_hotspots.csv ({len(hotspots)} rows)")
        
        if not mvp.empty:
            mvp.to_csv(f"{outdir}/qc_mvp_forecast.csv", index=False)
            print(f"Saved qc_mvp_forecast.csv ({len(mvp)} rows)")
            
        if not pro.empty:
            pro.to_csv(f"{outdir}/qc_pro_forecast.csv", index=False)
            print(f"Saved qc_pro_forecast.csv ({len(pro)} rows)")
            
        if not final.empty:
            final.to_csv(f"{outdir}/qc_final_forecast.csv", index=False)
            print(f"Saved qc_final_forecast.csv ({len(final)} rows)")
        
    except Exception as e:
        print(f"Save error: {e}")
    
    return final

# ---------------------------
# Main Execution
# ---------------------------
def run_forecast_pipeline(paths, outdir=".", params=None):
    """Main pipeline execution"""
    
    if params is None:
        params = PARAMS
        
    print("Starting QC Forecast Pipeline...")
    print("="*50)
    
    try:
        # Load data
        products, stores, sales, warranty, today = load_and_validate_data(paths)
        
        # A) Hotspot analysis
        print("Building hotspot ranking...")
        hotspots = build_hotspots(products, stores, sales, warranty, today, params)
        
        # B) MVP forecast  
        print("Generating MVP forecast...")
        mvp = mvp_forecast(products, stores, sales, warranty, today, params)
        
        # C) PRO forecast
        print("Generating PRO forecast...")
        pro = pro_forecast(products, stores, sales, warranty, today, params)
        
        # D) Blend and save
        print("Blending and saving results...")
        final = blend_and_save(hotspots, mvp, pro, params, outdir)
        
        # Detailed inventory allocation output
        print(f"\nDETAILED REFURBISH INVENTORY ALLOCATION")
        print(f"Target Date: {today + timedelta(days=params['HORIZON_DAYS'])}")
        print("="*80)
        
        if not final.empty:
            restock = final[final["final_units_to_prestore"] > 0].copy()
            
            if not restock.empty:
                # Sort by urgency (latest_stock_date) and quantity
                restock = restock.sort_values(["latest_stock_date", "final_units_to_prestore"], 
                                            ascending=[True, False])
                
                print("\nSTORE-BY-STORE ALLOCATION (Ship by Latest Stock Date):")
                print("-" * 80)
                
                current_store = ""
                store_total = 0
                
                for _, row in restock.iterrows():
                    store_key = f"{row['country']} - {row['store_name']}"
                    
                    # New store header
                    if store_key != current_store:
                        if current_store:  # Print previous store total
                            print(f"    └─ Store Total: {store_total} units\n")
                        
                        print(f"{store_key}")
                        print(f"   Ship by: {row['latest_stock_date']}")
                        current_store = store_key
                        store_total = 0
                    
                    # Product line
                    units = int(row['final_units_to_prestore'])
                    product = row['product_name'][:40]  # Truncate long names
                    risk_info = ""
                    if 'risk_rank' in row and pd.notna(row['risk_rank']):
                        risk_info = f" (Risk #{int(row['risk_rank'])})"
                    
                    print(f"   • {product:<40} → {units:3d} units{risk_info}")
                    store_total += units
                
                # Print last store total
                if current_store:
                    print(f"    └─ Store Total: {store_total} units\n")
                
                # Summary statistics
                total_units = restock["final_units_to_prestore"].sum()
                total_stores = restock["store_name"].nunique()
                total_products = restock["product_name"].nunique()
                total_countries = restock["country"].nunique()
                
                print(f"ALLOCATION SUMMARY:")
                print("-" * 40)
                print(f"Total Units to Ship:     {total_units:,}")
                print(f"Stores to Serve:         {total_stores}")
                print(f"Unique Products:         {total_products}")
                print(f"Countries:               {total_countries}")
                
                # Top products needing refurb
                top_products = (restock
                              .groupby("product_name")
                              .agg(total_needed=("final_units_to_prestore", "sum"),
                                   stores_affected=("store_name", "nunique"))
                              .reset_index()
                              .sort_values("total_needed", ascending=False)
                              .head(10))
                
                print(f"\n TOP 10 PRODUCTS NEEDING REFURB:")
                print("-" * 50)
                for _, row in top_products.iterrows():
                    product = row['product_name'][:35]
                    print(f"{product:<35} → {int(row['total_needed']):4d} units ({int(row['stores_affected'])} stores)")
                
                # Urgent shipments (next 7 days)
                urgent_date = today + timedelta(days=7)
                urgent = restock[pd.to_datetime(restock['latest_stock_date']) <= pd.to_datetime(urgent_date)]
                
                if not urgent.empty:
                    urgent_total = urgent["final_units_to_prestore"].sum()
                    urgent_stores = urgent["store_name"].nunique()
                    
                    print(f"\n⚡ URGENT SHIPMENTS (Next 7 Days):")
                    print("-" * 40)
                    print(f"Units needed ASAP:       {urgent_total:,}")
                    print(f"Stores requiring urgent: {urgent_stores}")
                    
                    # Show top urgent stores
                    urgent_by_store = (urgent
                                     .groupby(["country", "store_name"])
                                     .agg(urgent_units=("final_units_to_prestore", "sum"),
                                          ship_by=("latest_stock_date", "min"))
                                     .reset_index()
                                     .sort_values("urgent_units", ascending=False)
                                     .head(5))
                    
                    print(f"\nTop 5 Urgent Stores:")
                    for _, row in urgent_by_store.iterrows():
                        print(f"  {row['country']} - {row['store_name'][:25]:<25} → {int(row['urgent_units']):3d} units by {row['ship_by']}")
                
            else:
                print("All stores sufficiently stocked!")
        
        print(f"\n Pipeline completed successfully!")
        print(f"📁 Results saved in: {outdir}/")
        
        return final
        
    except Exception as e:
        print(f"Pipeline failed: {e}")
        raise

# ---------------------------
# Script Entry Point
# ---------------------------
if __name__ == "__main__":
    
    PATHS = {
    "products": "/Users/moonjiung/Desktop/Data analyst/Apple/Dataset/products.csv",
    "stores": "/Users/moonjiung/Desktop/Data analyst/Apple/Dataset/stores.csv",
    "sales": "/Users/moonjiung/Desktop/Data analyst/Apple/Dataset/sales.csv",
    "warranty": "/Users/moonjiung/Desktop/Data analyst/Apple/Dataset/warranty.csv"
}
    
    # Output directory
    OUTDIR = "."
    
    # Run pipeline
    final_forecast = run_forecast_pipeline(PATHS, outdir=OUTDIR, params=PARAMS)

Starting QC Forecast Pipeline...
 Data loaded. Sales: 1,040,191, Claims: 30,836, TODAY: 2024-08-21
Building hotspot ranking...
Generating MVP forecast...
Generating PRO forecast...
Blending and saving results...
Saved qc_hotspots.csv (204 rows)
Saved qc_mvp_forecast.csv (36 rows)
Saved qc_pro_forecast.csv (3100 rows)
Saved qc_final_forecast.csv (3100 rows)

DETAILED REFURBISH INVENTORY ALLOCATION
Target Date: 2024-09-20

STORE-BY-STORE ALLOCATION (Ship by Latest Stock Date):
--------------------------------------------------------------------------------
India - Apple New Delhi
   Ship by: 2024-09-06
   • Apple Watch Series 5                     →  13 units
   • iPhone 11 Pro                            →  13 units
   • Apple One                                →  12 units
   • iPhone 11 Pro Max                        →  12 units
    └─ Store Total: 50 units

Turkey - Apple Ankara
   Ship by: 2024-09-06
   • Apple Watch Series 5                     →  12 units
   • iPad (7th Gen)        